In [1]:
import astropy.units as u
import numpy as np
from astropy.io import ascii

import naima
from naima.models import ExponentialCutoffPowerLaw, InverseCompton, Synchrotron

## Read data

soft_xray1 = ascii.read("mos1+mos2_pwn_nh-corr.dat")
soft_xray2 = ascii.read("chandra_pwn_nh-corr.dat")
vhe = ascii.read("HAWC_J1826-128.dat")

## Model definition


def ElectronSynIC(pars, data):

    # Match parameters to ECPL properties, and give them the appropriate units
    amplitude = 10 ** pars[0] / u.eV
    alpha = pars[1]
    e_cutoff = (10 ** pars[2]) * u.TeV
    B = pars[3] * u.uG

    # Initialize instances of the particle distribution and radiative models
    ECPL = ExponentialCutoffPowerLaw(amplitude, 10.0 * u.TeV, alpha, e_cutoff)
    # Compute IC on CMB and on a FIR component with values from GALPROP for the
    # position of RXJ1713
    IC = InverseCompton(
        ECPL,
        seed_photon_fields=[
            "CMB",
            ["FIR", 9.5 * u.K, 0.5 * u.eV / u.cm ** 3],
        ],
        Eemin=1 * u.GeV,
    )
    SYN = Synchrotron(ECPL, B=B)

    # compute flux at the energies given in data['energy']
    model = IC.flux(data, distance=3.5 * u.kpc) + SYN.flux(
        data, distance=3.5 * u.kpc
    )

    # The first array returned will be compared to the observed spectrum for
    # fitting. All subsequent objects will be stored in the sampler metadata
    # blobs.
    return model, IC.compute_We(Eemin=1 * u.TeV)


## Prior definition


def lnprior(pars):
    """
    Return probability of parameter values according to prior knowledge.
    Parameter limits should be done here through uniform prior ditributions
    """
    # Limit norm and B to be positive
    logprob = (
        naima.uniform_prior(pars[0], 0.0, np.inf)
        + naima.uniform_prior(pars[1], -1, 5)
        + naima.uniform_prior(pars[3], 0, np.inf)
    )

    return logprob


if __name__ == "__main__":

    ## Set initial parameters and labels
    # Estimate initial magnetic field and get value in uG
    B0 = 2 * naima.estimate_B(soft_xray1, vhe).to("uG").value
    #B0 = 5

    p0 = np.array((33, 2.5, np.log10(48.0), B0))
    labels = ["log10(norm)", "index", "log10(cutoff)", "B"]

    ## Run sampler
    sampler, pos = naima.run_sampler(
        data_table=[soft_xray1, vhe],
        p0=p0,
        labels=labels,
        model=ElectronSynIC,
        prior=lnprior,
        nwalkers=32,
        nburn=100,
        nrun=20,
        threads=4,
        prefit=True,
        interactive=False,
    )

    ## Save run results to HDF5 file (can be read later with naima.read_run)
    naima.save_run("J1826_HAWC_SynIC", sampler)
    
    ## Diagnostic plots
    naima.save_diagnostic_plots(
        "J1826_HAWC_SynIC",
        sampler,
        sed=True,
        blob_labels=["Spectrum", "$W_e$($E_e>1$ TeV)"],
    )
    naima.save_results_table("J1826_HAWC_SynIC", sampler)
    
    plt = naima.plot_fit(sampler, label="Eel Pulsar Wind Nebula SED", sed=True, confs=[3,1], e_range=[0.00001*u.eV, 100*u.TeV])
    
    plt.savefig('J1826_SED_HAWC.pdf', format='pdf', bbox_inches='tight')
    
    plt.show()

INFO: Launching interactive model fitter, close when finished [naima.core]


<Figure size 640x480 with 9 Axes>

INFO: Finding Maximum Likelihood parameters through Nelder-Mead fitting... [naima.core]
INFO:    Initial parameters: [33.49329693  2.5         1.68124124  1.33046388] [naima.core]
INFO:    Initial lnprob(p0): -1590.844 [naima.core]
INFO:    New ML parameters : [32.59056012  2.81688479  2.37883462  0.93191256] [naima.core]
INFO:    Maximum lnprob(p0): -15.078 [naima.core]
Burning in the 32 walkers with 100 steps...

Progress of the run: 0 percent (0 of 100 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.6            2.82            2.38            0.931     
  Last ensemble std    :      0.109          0.0152          0.0115          0.00463    
  Last ensemble lnprob :  avg: -29.239, max: -15.186

Progress of the run: 5 percent (5 of 100 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.6            2.82            2.38     


Progress of the run: 5 percent (1 of 20 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.4            2.79            3.07            0.963     
  Last ensemble std    :      0.239           0.203           0.912           0.105     
  Last ensemble lnprob :  avg: -14.069, max: -11.621

Progress of the run: 10 percent (2 of 20 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.4            2.78            3.13            0.961     
  Last ensemble std    :      0.242           0.202           0.907           0.11      
  Last ensemble lnprob :  avg: -14.129, max: -11.621

Progress of the run: 15 percent (3 of 20 steps)
                           --log10(norm)-- -----index----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.4            2.78            3.12            0.935     
  Last ensemble std    :      

INFO: -------------------log10(norm)--------------------
          log10(norm) = $32.38^{+0.11}_{-0.3}$
                 norm = $(2.4^{+0.7}_{-1.1})\times10^{32}$ [naima.plot]
INFO: Plotting chain of parameter index... [naima.analysis]
INFO: ----------------------index-----------------------
          index = $2.79^{+0.05}_{-0.12}$ [naima.plot]
INFO: Plotting chain of parameter log10(cutoff)... [naima.analysis]
INFO: ------------------log10(cutoff)-------------------
          log10(cutoff) = $3.2^{+0.9}_{-0.5}$
                 cutoff = $(1.5^{+11}_{-1.1})\times10^{3}$ [naima.plot]
INFO: Plotting chain of parameter B... [naima.analysis]
INFO: ------------------------B-------------------------
          B = $0.96^{+0.13}_{-0.08}$ [naima.plot]
INFO: Plotting corner plot... [naima.analysis]
INFO: Plotting Spectrum... [naima.analysis]


/Users/danielburgess/anaconda3/lib/python3.7/site-packages/naima/analysis.py:160: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  f.clf()
/Users/danielburgess/anaconda3/lib/python3.7/site-packages/naima/analysis.py:160: UserWarning: Attempting to set identical left == right == 1.0 results in singular transformations; automatically expanding.
  f.clf()


INFO: Plotting $W_e$($E_e>1$ TeV)... [naima.analysis]
INFO: Saving results table in J1826_HAWC_SynIC_results.ecsv [naima.analysis]


/Users/danielburgess/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
